In [3]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from matplotlib import cm as CM
import numpy as NP
import tensorflow as tf

In [4]:
tokens = pd.read_csv (r'Traits.csv')
sales = pd.read_csv (r'Sales.csv')

In [5]:
tokens

,Unnamed: 0,collection,contract,token_id,nft_url,trait_type,value
0,0,azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,0,https://ikzttp.mypinata.cloud/ipfs/QmQFkLSQysj...,Type,Human
1,1,azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,0,https://ikzttp.mypinata.cloud/ipfs/QmQFkLSQysj...,Hair,Water
2,2,azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,0,https://ikzttp.mypinata.cloud/ipfs/QmQFkLSQysj...,Clothing,Pink Oversized Kimono
3,3,azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,0,https://ikzttp.mypinata.cloud/ipfs/QmQFkLSQysj...,Eyes,Striking
4,4,azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,0,https://ikzttp.mypinata.cloud/ipfs/QmQFkLSQysj...,Mouth,Frown
...,...,...,...,...,...,...,...
696003,44435,pudgy_penguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,8887,https://ipfs.io/ipfs/QmWXJXRdExse2YHRY21Wvh4pj...,Background,Blue
696004,44436,pudgy_penguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,8887,https://ipfs.io/ipfs/QmWXJXRdExse2YHRY21Wvh4pj...,Skin,Olive Green
696005,44437,pudgy_penguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,8887,https://ipfs.io/ipfs/QmWXJXRdExse2YHRY21Wvh4pj...,Body,Puffer Green
696006,44438,pudgy_penguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,8887,https://ipfs.io/ipfs/QmWXJXRdExse2YHRY21Wvh4pj...,Face,Handlebar Bear


In [6]:
print(pd.unique(tokens['token_id']))
print(tokens.groupby('collection')['token_id'].nunique())

[    0     1     2 ... 19997 19999 20000]
collection
BAYC              10000
MAYC              18502
azuki             10000
clone_x           18895
cool_cats          9933
doodles           10000
meebits           20000
pudgy_penguins     8888
Name: token_id, dtype: int64


In [7]:
grouped_tokens = tokens.groupby('collection')
bayc = grouped_tokens.get_group('BAYC')
mayc = grouped_tokens.get_group('MAYC')
azuki = grouped_tokens.get_group('azuki')
clone_x = grouped_tokens.get_group('clone_x')
cool_cats = grouped_tokens.get_group('cool_cats')
doodles = grouped_tokens.get_group('doodles')
meebits = grouped_tokens.get_group('meebits')
pudgy_penguins = grouped_tokens.get_group('pudgy_penguins')

In [8]:
grouped_sales = sales.groupby('tokenSymbol')
bayc_sales = grouped_sales.get_group('BAYC')
mayc_sales = grouped_sales.get_group('MAYC')
azuki_sales = grouped_sales.get_group('AZUKI')
clone_x_sales = grouped_sales.get_group('CloneX')
cool_cats_sales = grouped_sales.get_group('COOL')
doodles_sales = grouped_sales.get_group('DOODLE')
meebits_sales = grouped_sales.get_group('⚇')
pudgy_penguins_sales = grouped_sales.get_group('PPG')

In [9]:
# bayc
print(bayc.groupby('trait_type')['token_id'].nunique())
print(bayc.groupby('trait_type')['value'].nunique())
print(pd.unique(bayc.groupby('trait_type').get_group('Background')['value']))
print(bayc.groupby('trait_type').get_group('Background')['value'].value_counts()/10000)

trait_type
Background    10000
Clothes        8114
Earring        2977
Eyes          10000
Fur           10000
Hat            7744
Mouth         10000
Name: token_id, dtype: int64
trait_type
Background     8
Clothes       43
Earring        6
Eyes          23
Fur           19
Hat           36
Mouth         33
Name: value, dtype: int64
['Orange' 'Aquamarine' 'Purple' 'Blue' 'Army Green' 'Yellow' 'Gray'
 'New Punk Blue']
Purple           0.1291
Yellow           0.1283
Orange           0.1273
Aquamarine       0.1266
Army Green       0.1243
Blue             0.1242
New Punk Blue    0.1232
Gray             0.1170
Name: value, dtype: float64


In [10]:
sales[sales['amount'] != 0]

,blockNumber,timeStamp,hash,from,contractAddress,to,tokenID,tokenName,tokenSymbol,gasPrice,currency,amount,amountUsd
200,13992202,1642010411,0xe3034e18aa6fa00ac39ffb6904baf9570cc31c1716c5...,0x0000000000000000000000000000000000000000,0xed5af388653567af2f388e6224dc7c4b3241c544,0xdec08cb92a506b88411da9ba290f3694be223c26,200,Azuki,AZUKI,619374837712,ETH,1.000000,3384.137681
201,13992202,1642010411,0xe3034e18aa6fa00ac39ffb6904baf9570cc31c1716c5...,0x0000000000000000000000000000000000000000,0xed5af388653567af2f388e6224dc7c4b3241c544,0xdec08cb92a506b88411da9ba290f3694be223c26,201,Azuki,AZUKI,619374837712,ETH,1.000000,3384.137681
202,13992202,1642010411,0xe3034e18aa6fa00ac39ffb6904baf9570cc31c1716c5...,0x0000000000000000000000000000000000000000,0xed5af388653567af2f388e6224dc7c4b3241c544,0xdec08cb92a506b88411da9ba290f3694be223c26,202,Azuki,AZUKI,619374837712,ETH,1.000000,3384.137681
203,13992202,1642010411,0xe3034e18aa6fa00ac39ffb6904baf9570cc31c1716c5...,0x0000000000000000000000000000000000000000,0xed5af388653567af2f388e6224dc7c4b3241c544,0xdec08cb92a506b88411da9ba290f3694be223c26,203,Azuki,AZUKI,619374837712,ETH,1.000000,3384.137681
204,13992202,1642010411,0xe3034e18aa6fa00ac39ffb6904baf9570cc31c1716c5...,0x0000000000000000000000000000000000000000,0xed5af388653567af2f388e6224dc7c4b3241c544,0xdec08cb92a506b88411da9ba290f3694be223c26,204,Azuki,AZUKI,619374837712,ETH,1.000000,3384.137681
...,...,...,...,...,...,...,...,...,...,...,...,...,...
431366,14471743,1648430919,0xb362fed9aece8fa0ce819778062ddbb05533a68a6401...,0x88341d1a8f672d2780c8dc725902aae72f143b0c,0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b,0xf6a325cef74fb60a685bdd5a79ded1ffe45996e8,550,CloneX,CloneX,23623743142,ETH,5.745767,18978.246953
431368,14471918,1648433160,0xcf8b9cf7a66f81ca0ca7648e4afa9183f171fb286bd8...,0xf6a325cef74fb60a685bdd5a79ded1ffe45996e8,0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b,0x88341d1a8f672d2780c8dc725902aae72f143b0c,550,CloneX,CloneX,25970855725,ETH,11.000000,36254.122921
431378,14472353,1648438868,0xb7e933b307f2aee8795311d6226e3f656aeee5d0683a...,0x23324ed44904260fe555b18e5ba95c6030b9227d,0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b,0xac7ebc289354503e1ddda06f30e75ca688382051,11018,CloneX,CloneX,38783642325,ETH,14.990000,49689.068827
431396,14473259,1648451159,0x695045f3093a9ee40769135159118f1fc2bd603a22a7...,0x24415add68ea0d8c6efa9ff6e4ca213417969d38,0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b,0x2310abf1ef6beb30b74bff98ac1b326b647f4224,911,CloneX,CloneX,23543438533,ETH,16.000000,53186.027771


In [11]:
print(sales.groupby('tokenSymbol')['tokenID'].nunique())

tokenSymbol
AZUKI     10000
BAYC      10000
COOL       9938
CloneX    18554
DOODLE     9784
MAYC      17990
PPG        8887
⚇         19999
Name: tokenID, dtype: int64


In [12]:
bayc_features = bayc.groupby('token_id')#.filter('trait_type')
print(bayc_features)

In [13]:
def get_features(df):
    """
    :param df: dataframe to have features extracted
    :return: dataframe with rows of token id and columns of features
    """
    columns = {}

    for trait in df['trait_type'].unique():
        columns[trait] = ['None' for i in range(df.token_id.nunique())]

    for id in df['token_id'].unique():
        for _, row in df[df.token_id == id].iterrows():
                columns[row.trait_type][id] = row.value
    #%%

    return pd.DataFrame(columns)

In [159]:
def process_sales(df):
    df = df[df.amount != 0]
    df = df.sort_values(['tokenID', 'timeStamp'], ascending=[True, False]).drop_duplicates('tokenID')
    print(df['tokenID'], df['timeStamp'], df['amount'])
    df = df[df.amount < 1e4]
    return df

In [160]:
print(process_sales(bayc_sales)['tokenID'])

93872        6
65561        9
98503       10
93873       11
101833      13
          ... 
101886    9995
59084     9996
77268     9997
80372     9998
51534     9999
Name: tokenID, Length: 9985, dtype: int64 93872     1640201192
65561     1623398673
98503     1643770560
93873     1640201192
101833    1647468884
             ...    
101886    1647477024
59084     1621664553
77268     1628068878
80372     1629471251
51534     1619858117
Name: timeStamp, Length: 9985, dtype: int64 93872       0.0025
65561       2.0050
98503       0.0050
93873       0.0025
101833     99.0000
            ...   
101886    105.0000
59084       0.3900
77268      20.0000
80372      21.2000
51534       0.0800
Name: amount, Length: 9985, dtype: float64
93872        6
65561        9
98503       10
93873       11
101833      13
          ... 
101886    9995
59084     9996
77268     9997
80372     9998
51534     9999
Name: tokenID, Length: 9970, dtype: int64


# BAYC

In [161]:
bayc_sales_processed = process_sales(bayc_sales)
token_ids = bayc_sales_processed['tokenID'].unique()
prices = bayc_sales_processed.groupby('tokenID')['amount'].max()
print(prices.describe())
print(bayc_sales_processed)
# print(token_ids)

93872        6
65561        9
98503       10
93873       11
101833      13
          ... 
101886    9995
59084     9996
77268     9997
80372     9998
51534     9999
Name: tokenID, Length: 9985, dtype: int64 93872     1640201192
65561     1623398673
98503     1643770560
93873     1640201192
101833    1647468884
             ...    
101886    1647477024
59084     1621664553
77268     1628068878
80372     1629471251
51534     1619858117
Name: timeStamp, Length: 9985, dtype: int64 93872       0.0025
65561       2.0050
98503       0.0050
93873       0.0025
101833     99.0000
            ...   
101886    105.0000
59084       0.3900
77268      20.0000
80372      21.2000
51534       0.0800
Name: amount, Length: 9985, dtype: float64
count    9970.000000
mean       30.569627
std        50.452453
min         0.000100
25%         0.490000
50%         4.900000
75%        50.000000
max      1080.690000
Name: amount, dtype: float64
        blockNumber   timeStamp  \
93872      13856860  1640201192   

In [162]:
bayc_features = get_features(bayc)

In [163]:
print(bayc_features)
print(bayc_features.iloc[0])

          Earring     Background           Fur           Clothes  \
0     Silver Hoop         Orange         Robot       Striped Tee   
1            None         Orange         Robot    Vietnam Jacket   
2            None     Aquamarine         Robot              None   
3            None         Purple       Cheetah     Bone Necklace   
4            None           Blue  Golden Brown  Navy Striped Tee   
...           ...            ...           ...               ...   
9995         None           Gray          Pink    Smoking Jacket   
9996  Silver Hoop  New Punk Blue    Dark Brown         Guayabera   
9997         None         Purple         Black      Sailor Shirt   
9998         None         Yellow         Brown        Bayc T Red   
9999    Gold Hoop         Purple          Gray              None   

                         Mouth        Eyes                Hat  
0                   Discomfort      X Eyes               None  
1                         Grin  Blue Beams             

In [164]:
def price_querrry(token_id, prices=bayc_sales_processed, features=bayc_features):
    out = []  # list of prices per category
    frequency_percent = []
    num_tokens = len(features.index)
    for (colname,colval) in features.iloc[token_id].iteritems():
        print(colname, ": ", colval)
        similar_tokens = features[features[colname] == colval]
        frequency_percent.append(features[colname].value_counts()[colval]/num_tokens*100)
        similar_ids = similar_tokens.index.tolist()
        out.append(prices.loc[prices['tokenID'].isin(similar_ids)]['amount'].mean())

    print(frequency_percent)
    return out#sum(out) / len(out)

In [165]:
price_querrry(2121)

Earring :  None
Background :  Blue
Fur :  Cream
Clothes :  Lab Coat
Mouth :  Phoneme Vuh
Eyes :  Holographic
Hat :  Halo
[70.23, 12.42, 6.36, 1.44, 3.3300000000000005, 1.51, 3.2399999999999998]


[30.1071454455799,
 28.127228684803388,
 31.405344305631445,
 26.19711098957495,
 29.991701872423025,
 32.350109584918286,
 30.05079965824389]

In [166]:
bayc_sales[(bayc_sales['tokenID'] == 2121)]# && (bayc_sales['amount'] != 0)]
# print(bayc_features.iloc[2121])

,blockNumber,timeStamp,hash,from,contractAddress,to,tokenID,tokenName,tokenSymbol,gasPrice,currency,amount,amountUsd
43502,12346736,1619851391,0xf5159c28c76d3650bf3a9741a72acc65b699d2713c4b...,0x0000000000000000000000000000000000000000,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x090c847ae5623cb7c538be34fe470b0c824b181f,2121,BoredApeYachtClub,BAYC,39000000000,ETH,0.080,228.905656
56745,12367810,1620132151,0x1c29147e0f9c424968bb63303a9a52da12b617dfb95a...,0x090c847ae5623cb7c538be34fe470b0c824b181f,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x7b15e6c439b27a553b65a9904ce571da6691a0fb,2121,BoredApeYachtClub,BAYC,43000000000,ETH,1.500,5198.710792
81424,13084487,1629760544,0xa9441e4f6ab15b70c1b7881f41b734574fb2a91b8173...,0x7b15e6c439b27a553b65a9904ce571da6691a0fb,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0xb7692c097a20f6767e0464bbe958d69bf082baf5,2121,BoredApeYachtClub,BAYC,72000000000,ETH,27.825,92727.061558
81426,13084521,1629761017,0x13a6358df62d303964a59fb7f874fdeb16fadaf42b16...,0xb7692c097a20f6767e0464bbe958d69bf082baf5,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0xd3fc6fec4b219c2d74b366fee6b585df71611533,2121,BoredApeYachtClub,BAYC,81000000000,ETH,0.000,0.000000
85327,13238300,1631815769,0x0a56f7f210701a1d100acd5b776d849e9975c6b5b64b...,0xd3fc6fec4b219c2d74b366fee6b585df71611533,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0xd27b7c6bd93f604bcd3497565f023652c2d13ce0,2121,BoredApeYachtClub,BAYC,136000000000,ETH,62.690,223233.123143
100968,14318159,1646366250,0x019261e503027c3632469d18da634ae7da4341726008...,0xd27b7c6bd93f604bcd3497565f023652c2d13ce0,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x4a8bdd15bdb8b1ab534d39c9f8d65a36011029f4,2121,BoredApeYachtClub,BAYC,47256082362,ETH,0.000,0.000000


# AZUKI

In [110]:
azuki_sales_processed = process_sales(azuki_sales)
token_ids = azuki_sales_processed['tokenID'].unique()
prices = azuki_sales_processed.groupby('tokenID')['amount'].max()
print(prices.describe())
print(azuki_sales_processed)
# print(token_ids)

32755      58
33086      98
29836     200
201       201
40944     202
         ... 
26298    9995
18086    9996
36759    9997
23115    9998
18341    9999
Name: tokenID, Length: 9802, dtype: int64 32755    1644523193
33086    1644729063
29836    1643693632
201      1642010411
40944    1648333308
            ...    
26298    1643181646
18086    1642669429
36759    1646271800
23115    1642828366
18341    1642700044
Name: timeStamp, Length: 9802, dtype: int64 32755    11.890000
33086    11.825000
29836    15.273333
201       1.000000
40944    19.900000
           ...    
26298     6.990000
18086     0.500000
36759    19.350000
23115    25.000000
18341     0.500000
Name: amount, Length: 9802, dtype: float64
count    9802.000000
mean        9.332306
std        10.840894
min         0.000010
25%         2.640000
50%         7.770000
75%        13.500000
max       204.000000
Name: amount, dtype: float64
       blockNumber   timeStamp  \
32755     14180331  1644523193   
33086     14195778  164

In [111]:
azuki_features = get_features(azuki)

In [112]:
print(azuki_features)
print(azuki_features.iloc[0])


       Type               Hair               Clothing         Eyes  \
0     Human              Water  Pink Oversized Kimono     Striking   
1     Human      Pink Hairband   White Qipao with Fur  Daydreaming   
2     Human         Pink Flowy                   Vest         Ruby   
3     Human        Green Spiky           Green Yukata     Careless   
4     Human   Brown Dreadlocks   White Qipao with Fur    Lightning   
...     ...                ...                    ...          ...   
9995  Human       Magenta Long           Black Kimono   Suspicious   
9996  Human  Blonde Swept Back           Black Kimono      Pensive   
9997  Human        Brown Messy     Azuki Track Jacket      Curious   
9998  Human  Orange Swept Back   Suit with Turtleneck       Closed   
9999  Human   Blue Short Spiky            Dress Shirt       Closed   

               Mouth            Offhand   Background         Ear  \
0              Frown  Monkey King Staff  Off White A        None   
1           Lipstick   

In [140]:
price_querrry(2121, azuki_sales_processed, azuki_features)

Type :  Human
Hair :  Water
Clothing :  Pink Oversized Kimono
Eyes :  Striking
Mouth :  Frown
Offhand :  Monkey King Staff
Background :  Off White A
Ear :  None
Headgear :  None
Neck :  None
Face :  None
Special :  None
[90.18, 0.21, 0.7100000000000001, 4.08, 3.63, 1.4500000000000002, 18.14, 81.81, 65.12, 77.46, 67.9, 93.71000000000001]


[8.74819985567065,
 28.940500000000007,
 7.532605830867647,
 11.205717240064535,
 8.87760524130392,
 8.389088911159263,
 8.882410868026527,
 9.220822510312129,
 9.267974513733474,
 9.215779040227359,
 9.36743621419964,
 8.775498077069088]

In [118]:
azuki_sales[(azuki_sales['tokenID'] == 2121)]# && (bayc_sales['amount'] != 0)]
# print(bayc_features.iloc[2121])


,blockNumber,timeStamp,hash,from,contractAddress,to,tokenID,tokenName,tokenSymbol,gasPrice,currency,amount,amountUsd
2121,13992207,1642010454,0xefcaf91ec92b6ca19151ae50b3a1af9180c8c1faaf32...,0x0000000000000000000000000000000000000000,0xed5af388653567af2f388e6224dc7c4b3241c544,0xae545300740d789791e998c2e655fdafe6341690,2121,Azuki,AZUKI,777000000000,ETH,1.00000,3384.137681
14884,14005196,1642184280,0x3881768fd13355d09d21aa78f04bab9aa0186d7f1f95...,0xae545300740d789791e998c2e655fdafe6341690,0xed5af388653567af2f388e6224dc7c4b3241c544,0xbad2e065edc215cb54a0e78391fb2abcb173f77d,2121,Azuki,AZUKI,200652041196,ETH,2.79838,9181.602635
29924,14118533,1643696582,0xb680279834465bb06e33e9e1af6b8bf97a92506a42b7...,0xbad2e065edc215cb54a0e78391fb2abcb173f77d,0xed5af388653567af2f388e6224dc7c4b3241c544,0x421a5b097b71060113c435e3feee236efba510dc,2121,Azuki,AZUKI,101411628574,ETH,14.49000,39595.642086
